# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [23]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory

### YOUR CODE HERE ###
from azureml.core.dataset import Dataset
##url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
##dataset2 = TabularDatasetFactory.from_delimited_files(path = url)
###ds2 = dataset2.to_pandas_dataframe() ### YOUR CODE HERE ###

url = 'https://raw.githubusercontent.com/zaza107-1/nd00333-capstone/master/heart_failure_clinical_records_dataset.csv'
dataset = Dataset.Tabular.from_delimited_files(path=url)
ds = dataset.to_pandas_dataframe()

ds.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [20]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'AutoML_heart'
project_folder = '.'
experiment=Experiment(ws, experiment_name)

In [10]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [4]:
from train2 import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds) ### your code###

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.33)

df_train = pd.concat([x_train, y_train], axis=1)

In [7]:
df_train.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,200.000000,200.000000,200.000000,200.000000,200.00000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,60.588335,0.420000,582.765000,0.395000,38.59500,0.365000,260130.062100,1.385200,136.620000,0.635000,0.305000,133.735000,0.270000
std,11.287966,0.494797,944.276208,0.490077,11.68003,0.482638,91079.331644,1.019065,4.318431,0.482638,0.461563,77.612973,0.445074
min,40.000000,0.000000,23.000000,0.000000,20.00000,0.000000,62000.000000,0.500000,113.000000,0.000000,0.000000,4.000000,0.000000
25%,52.000000,0.000000,109.000000,0.000000,30.00000,0.000000,203000.000000,0.900000,134.000000,0.000000,0.000000,78.750000,0.000000
50%,60.000000,0.000000,244.500000,0.000000,38.00000,0.000000,256000.000000,1.100000,137.000000,1.000000,0.000000,119.500000,0.000000
75%,67.250000,1.000000,582.000000,1.000000,45.00000,1.000000,304250.000000,1.400000,140.000000,1.000000,1.000000,206.250000,1.000000
max,95.000000,1.000000,7861.000000,1.000000,80.00000,1.000000,621000.000000,9.000000,146.000000,1.000000,1.000000,285.000000,1.000000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [19]:
compute_target

AmlCompute(workspace=Workspace.create(name='quick-starts-ws-136737', subscription_id='9e65f93e-bdd8-437b-b1e8-0647cd6098f7', resource_group='aml-quickstarts-136737'), name=auto-ml, id=/subscriptions/9e65f93e-bdd8-437b-b1e8-0647cd6098f7/resourceGroups/aml-quickstarts-136737/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-136737/computes/auto-ml, type=AmlCompute, provisioning_state=Succeeded, location=southcentralus, tags=None)

In [27]:
if 'training' not in os.listdir():
    os.mkdir("./training")

In [28]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 10,
                    "max_concurrent_iterations": 5,
                    "primary_metric" : 'Accuracy'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [29]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

remote_run.wait_for_completion()

ConfigException: ConfigException:
	Message: Input of type 'Unknown' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset, azureml.pipeline.core.pipeline_output_dataset.PipelineOutputTabularDataset]
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Input of type 'Unknown' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset, azureml.pipeline.core.pipeline_output_dataset.PipelineOutputTabularDataset]",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "training_data",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid",
                "inner_error": {
                    "code": "InvalidInputDatatype"
                }
            }
        }
    }
}

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

In [ ]:
#TODO: Save the best model

best_run.register_model(model_name = 'automl_best_model.pkl', model_path = './outputs/')

import joblib

joblib.dump(value= fitted_model, filename = 'outputs/model_ML_fitted.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.delete()